<a href="https://colab.research.google.com/github/luanps/pyserini/blob/master/Pyserini_BM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pyserini: BM25 Baseline for MS MARCO Passage Retrieval

### Install dependencies

In [15]:
from google.colab import auth
auth.authenticate_user()

In [1]:
!git clone https://github.com/luanps/pyserini.git

Cloning into 'pyserini'...
remote: Enumerating objects: 4366, done.
remote: Total 4366 (delta 0), reused 0 (delta 0), pack-reused 4366
Receiving objects: 100% (4366/4366), 2.13 MiB | 17.86 MiB/s, done.
Resolving deltas: 100% (3001/3001), done.


In [2]:
!pip install pyserini

     |████████████████████████████████| 84.6 MB 98 kB/s 
     |████████████████████████████████| 13.5 MB 44.3 MB/s 
     |████████████████████████████████| 3.5 MB 45.5 MB/s 
     |████████████████████████████████| 1.1 MB 33.2 MB/s 
     |████████████████████████████████| 4.9 MB 38.4 MB/s 
     |████████████████████████████████| 1.2 MB 58.4 MB/s 
     |████████████████████████████████| 2.0 MB 55.0 MB/s 
     |████████████████████████████████| 6.0 MB 40.1 MB/s 
     |████████████████████████████████| 188 kB 48.1 MB/s 
     |████████████████████████████████| 451 kB 34.3 MB/s 
     |████████████████████████████████| 181 kB 59.0 MB/s 
     |████████████████████████████████| 628 kB 59.7 MB/s 
     |████████████████████████████████| 10.1 MB 44.8 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 895 kB 41.2 MB/s 
     |████████████████████████████████| 6.5 MB 40.7 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |███████████

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

!pip install faiss-cpu

     |████████████████████████████████| 8.6 MB 7.1 MB/s 


### Download and Preproc MSMARCO dataset

In [4]:
#Download MSMARCO dataset

!mkdir collections/msmarco-passage
!wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz -P collections/msmarco-passage
# Alternative mirror:
# wget https://www.dropbox.com/s/9f54jg2f71ray3b/collectionandqueries.tar.gz -P collections/msmarco-passage
!tar xvfz collections/msmarco-passage/collectionandqueries.tar.gz -C collections/msmarco-passage

mkdir: cannot create directory ‘collections/msmarco-passage’: No such file or directory
--2022-03-02 16:29:40--  https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1057717952 (1009M) [application/gzip]
Saving to: ‘collections/msmarco-passage/collectionandqueries.tar.gz’

collectionandquerie 100%[===================>]   1009M  18.4MB/s    in 56s     

2022-03-02 16:30:37 (17.9 MB/s) - ‘collections/msmarco-passage/collectionandqueries.tar.gz’ saved [1057717952/1057717952]

collection.tsv
qrels.dev.small.tsv
qrels.train.tsv
queries.dev.small.tsv
queries.dev.tsv
queries.eval.small.tsv
queries.eval.tsv
queries.train.tsv


In [ ]:
# Convert MSMARCO to jsonl
!python pyserini/scripts/ltr_msmarco/convert_collection_to_jsonl.py  --collection-path collections/msmarco-passage/collection.tsv  --output-folder collections/msmarco-passage/collection_jsonl

In [7]:
# Index Collection
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input collections/msmarco-passage/collection_jsonl \
  --index indexes/lucene-index-msmarco-passage \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2022-03-02 16:41:31,511 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Setting log level to INFO
2022-03-02 16:41:31,516 INFO  [main] index.IndexCollection (IndexCollection.java:646) - Starting indexer...
2022-03-02 16:41:31,516 INFO  [main] index.IndexCollection (IndexCollection.java:647) - ============ Loading Parameters ============
2022-03-02 16:41:31,517 INFO  [main] index.IndexCollection (IndexCollection.java:648) - DocumentCollection path: collections/msmarco-passage/collection_jsonl
2022-03-02 16:41:31,517 INFO  [main] index.IndexCollection (IndexCollection.java:649) - CollectionClass: JsonCollection
2022-03-02 16:41:31,518 INFO  [main] index.IndexCollection (IndexCollection.java:650) - Generator: DefaultLuceneDocumentGenerator
2022-03-02 16:41:31,518 INFO  [main] index.IndexCollection (IndexCollection.java:651) - Threads: 9
2022-03-02 16:41:31,519 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Language: en
2022-03-02 16:41:31,519 INFO  [main] 

### Run Inference

In [8]:
!python -m pyserini.search.lucene \
  --index msmarco-passage \
  --topics msmarco-passage-dev-subset \
  --output run.msmarco-passage.bm25tuned.txt \
  --output-format msmarco \
  --hits 1000 \
  --bm25 --k1 0.82 --b 0.68

Using pre-defined topic order for msmarco-passage-dev-subset
Attempting to initialize pre-built index msmarco-passage.
index-msmarco-passage-20201117-f87c94.tar.gz: 2.07GB [02:45, 13.4MB/s]                
Extracting /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.tar.gz into /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.1efad4f1ae6a77e235042eff4be1612d...
Initializing msmarco-passage...
Setting BM25 parameters: k1=0.82, b=0.68
Running msmarco-passage-dev-subset topics, saving to run.msmarco-passage.bm25tuned.txt...
100% 6980/6980 [14:06<00:00,  8.25it/s]


In [11]:
#MRR Eval
!python -m pyserini.eval.msmarco_passage_eval collections/msmarco-passage/qrels.dev.small.tsv run.msmarco-passage.bm25tuned.txt >> bm25_mrr_eval.txt


In [13]:
#TREC Eval
!python -m pyserini.eval.convert_msmarco_run_to_trec_run --input run.msmarco-passage.bm25tuned.txt \
                                                         --output run.msmarco-passage.bm25tuned.trec

!python -m pyserini.eval.trec_eval -c -mrecall.1000 \
                                      -mmap msmarco-passage-dev-subset \
                                      run.msmarco-passage.bm25tuned.trec >> bm25_trec_eval.txt

jtreceval-0.0.5-jar-with-dependencies.jar: 1.79MB [00:00, 4.39MB/s]                


In [16]:
!gsutil cp bm25_trec_eval.txt bm25_mrr_eval.txt run.msmarco-passage.bm25tuned.txt gs://luanps/information_retrieval/pyserini/bm25/

Copying file://bm25_trec_eval.txt [Content-Type=text/plain]...
Copying file://bm25_mrr_eval.txt [Content-Type=text/plain]...
Copying file://run.msmarco-passage.bm25tuned.txt [Content-Type=text/plain]...
/
Operation completed over 3 objects/126.9 MiB.                                    
